In [2]:
import pandas as pd
import os

# Define the path to your data file
file_path = '../Supply chain logistics problem.xlsx'

# List of sheet names you want to load from the Excel file
sheet_names = [
    "OrderList",
    "FreightRates",
    "WhCosts",
    "WhCapacities",
    "ProductsPerPlant",
    "VmiCustomers",
    "PlantPorts"
]

# Load all sheets into a dictionary of DataFrames
dfs = {}
try:
    for sheet in sheet_names:
        # The key in the dictionary will be the lowercase sheet name (e.g., 'orderlist')
        dfs[sheet.lower()] = pd.read_excel(file_path, sheet_name=sheet)
        print(f"Successfully loaded sheet: {sheet}")
except Exception as e:
    print(f"An error occurred: {e}")

# Now you can access each dataframe to verify it loaded correctly
# For example, to see the first 5 rows of the OrderList data:
if 'orderlist' in dfs:
    print("\n--- Sample of OrderList Data ---")
    print(dfs['orderlist'].head())

Successfully loaded sheet: OrderList
Successfully loaded sheet: FreightRates
Successfully loaded sheet: WhCosts
Successfully loaded sheet: WhCapacities
Successfully loaded sheet: ProductsPerPlant
Successfully loaded sheet: VmiCustomers
Successfully loaded sheet: PlantPorts

--- Sample of OrderList Data ---
       Order ID Order Date Origin Port Carrier  TPT Service Level  \
0  1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
1  1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
2  1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
3  1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
4  1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   

   Ship ahead day count  Ship Late Day count   Customer  Product ID  \
0                     3                    0  V55555_53     1700106   
1                     3                    0  V55555_53     1700106   
2                     3                    0  V55555_53     1700

In [9]:
# --- 1. Calculate Total Demand Per Product ---
# Group the order list by 'Product ID' and sum the 'Unit quantity' for each product.
product_demand = dfs['orderlist'].groupby('Product ID')['Unit quantity'].sum().reset_index()
product_demand.rename(columns={'Unit quantity': 'Total_Demand'}, inplace=True)

# --- 2. Identify Sole-Sourced Products ---
# First, count how many unique plants can produce each product.
sourcing_counts = dfs['productsperplant'].groupby('Product ID')['Plant Code'].nunique().reset_index()
sourcing_counts.rename(columns={'Plant Code': 'Num_Sourcing_Plants'}, inplace=True)

# Now, filter to find products that are only made by a single plant.
sole_sourced_products = sourcing_counts[sourcing_counts['Num_Sourcing_Plants'] == 1]

# --- 3. Combine Demand and Sourcing Information ---
# Merge the demand data with the list of sole-sourced products.
bottleneck_analysis = pd.merge(sole_sourced_products, product_demand, on='Product ID')

# Merge with the original productsperplant table to find out *which* specific plant is the sole source.
bottleneck_analysis = pd.merge(bottleneck_analysis, dfs['productsperplant'], on='Product ID')

# --- 4. Add Plant Capacity and Analyze ---
# Merge with the capacities table to get the daily capacity of the sourcing plant.
capacities_renamed = dfs['whcapacities'].rename(columns={'Plant ID': 'Plant Code'})
bottleneck_analysis = pd.merge(bottleneck_analysis, capacities_renamed, on='Plant Code')

# Calculate the ratio of demand to capacity and create a new column.
bottleneck_analysis['Demand_vs_Capacity_Ratio'] = bottleneck_analysis['Total_Demand'] / bottleneck_analysis['Daily Capacity ']

# --- 5. Display the Most Critical Bottlenecks ---
# Sort the results to show the highest, most critical ratios first.
critical_bottlenecks = bottleneck_analysis.sort_values(by='Demand_vs_Capacity_Ratio', ascending=False)

print("--- Analysis of Potential Supply Chain Bottlenecks (Sole-Sourced Products) ---")
display(critical_bottlenecks.head(15)) # Display the top 15 most critical items

--- Analysis of Potential Supply Chain Bottlenecks (Sole-Sourced Products) ---


,Product ID,Num_Sourcing_Plants,Total_Demand,Plant Code,Daily Capacity,Demand_vs_Capacity_Ratio
323,1684862,1,3470409,PLANT03,1013,3425.872655
222,1676592,1,1119252,PLANT03,1013,1104.888450
596,1700569,1,904493,PLANT03,1013,892.885489
114,1664051,1,873011,PLANT03,1013,861.807502
136,1667927,1,814076,PLANT03,1013,803.628825
587,1700140,1,811381,PLANT03,1013,800.968411
305,1683560,1,772283,PLANT03,1013,762.372162
127,1666524,1,670437,PLANT03,1013,661.833169
583,1700130,1,658352,PLANT03,1013,649.903258
590,1700143,1,654556,PLANT03,1013,646.155972


In [10]:
# --- 1. Isolate a Specific Shipping Lane for Analysis ---
# We'll look at the rates for carrier V444_6 on the route from PORT08 to PORT09.
freight_rates = dfs['freightrates'].copy()

example_lane = freight_rates[
    (freight_rates['orig_port_cd'] == 'PORT08') &
    (freight_rates['dest_port_cd'] == 'PORT09') &
    (freight_rates['Carrier'] == 'V444_6')
].copy() # Use.copy() to avoid SettingWithCopyWarning

# Sort by the minimum weight to see the tiers clearly
example_lane.sort_values(by='minm_wgh_qty', inplace=True)


# --- 2. Display the Tiered Rate Card ---
print("--- Tiered Freight Rate Card for Carrier V444_6 (PORT08 to PORT09) ---")
display(example_lane[['minm_wgh_qty', 'max_wgh_qty', 'rate', 'minimum cost']])


# --- 3. Explain the Findings ---
print("\n--- Key Findings from Freight Rate Analysis ---")
print("1. Tiered Pricing: Notice that the 'rate' (cost per unit of weight) is not constant. It decreases as the shipment gets heavier. For example, a small shipment (10-15 units) is charged at a rate of 1.8272, while a very large one (>2000 units) is charged at a much lower rate of 0.6372.")
print("\n2. Minimum Cost Rule: Every shipment on this lane is subject to a 'minimum cost' of 43.2272. The final freight cost is the HIGHER of (weight * rate) or the minimum cost.")
print("\nThis non-linear pricing is too complex for a standard optimization model. This is why we will first train a machine learning model to predict these costs, turning this complex table into simple cost estimates.")

--- Tiered Freight Rate Card for Carrier V444_6 (PORT08 to PORT09) ---


,minm_wgh_qty,max_wgh_qty,rate,minimum cost
12,0.0,4.99,1.8272,43.2272
13,5.0,9.99,1.8272,43.2272
11,10.0,14.99,1.8272,43.2272
18,15.0,19.99,1.5872,43.2272
16,20.0,24.99,1.3212,43.2272
8,25.0,29.99,1.2452,43.2272
9,30.0,34.99,1.1312,43.2272
4,35.0,39.99,1.0552,43.2272
17,40.0,44.99,0.9792,43.2272
15,45.0,49.99,0.9412,43.2272



--- Key Findings from Freight Rate Analysis ---
1. Tiered Pricing: Notice that the 'rate' (cost per unit of weight) is not constant. It decreases as the shipment gets heavier. For example, a small shipment (10-15 units) is charged at a rate of 1.8272, while a very large one (>2000 units) is charged at a much lower rate of 0.6372.

2. Minimum Cost Rule: Every shipment on this lane is subject to a 'minimum cost' of 43.2272. The final freight cost is the HIGHER of (weight * rate) or the minimum cost.

This non-linear pricing is too complex for a standard optimization model. This is why we will first train a machine learning model to predict these costs, turning this complex table into simple cost estimates.
